In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os 
import sys
from datetime import datetime, timedelta
from modeling.utils import process_address
from modeling.utils import * 
from collections import Counter
import warnings
import json 
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

In [11]:
target = ["price"]
dense_features = ["LON", 
                "LAT", 
                "building_sqft", 
                "Lot Size", 
                "Year Built", 
                "Garage Number", 
                "Bedrooms", 
                "Baths", 
                "Maintenance Fee", 
                "Tax Rate", 
                "Recent Market Value", 
                "Recent Tax Value"]
cate = ["status", "Property Type", "County", "Private Pool", "Area Pool"]
time_col = ["date"]
cate_multi = ["Foundation_multiclass", "Garage Types_multiclass", 
            "Roof Type_multiclass", "Pool_feature_multiclass", "floor_type_multiclass", 
            "finance_option_multiclass", "Exterior Type_multiclass", "Exterior_multiclass", 
            "Style_multiclass"]

add_numerial_features = ["elementary_school_star", "middle_school_star", "high_school_star"]
add_cate_multi = ["school_org"] # ["school_names", "school_grades", "school_org"]

# df = pd.read_pickle("data/property_structured.pkl")
# df.head(2)

In [12]:
raw_df = pd.read_pickle("data/contracted_houses_11052023_11062024.pkl")
raw_df.head(3)

,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date,json_path,time_epoch,image_path,json_size,image_cnt
57,"4513 Refugio Dr, Plano, TX 75024",https://www.har.com//homedetail/4513-refugio-d...,"4 bedrooms 3,395 Sqft. ($233/Sqft.) 3 full & 1...",Karla Davis Fathom Realty,789990.0,66.0,Under Contract - P,75024,4.0,0.0,0.0,3395.0,0.0,0.0,0.0,"4513 Refugio Dr, Plano, TX 75024",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1690411643,/home/user/DataCenter/HAR_data/house_images/00...,4425,12
8,"1516 Bay Area Blvd P12, Houston, TX 77058",https://www.har.com//homedetail/1516-bay-area-...,1 bedrooms 684 Sqft. ($117/Sqft.) 1 full baths...,Laurie Long RE/MAX Space Center,79900.0,79.0,Under Contract - PS,77058,1.0,0.0,0.0,684.0,0.0,0.0,0.0,"1516 Bay Area Blvd P12, Houston, TX 77058",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1689375459,/home/user/DataCenter/HAR_data/house_images/00...,5661,11
22,"1717 San Sebastian Ln, Houston, TX 77058",https://www.har.com//homedetail/1717-san-sebas...,"2 bedrooms 1,280 Sqft. ($148/Sqft.) 2 full & 1...",Austin Jones Keller Williams Realty Clear ...,189000.0,10.0,Under Contract - P,77058,2.0,0.0,0.0,1280.0,0.0,0.0,0.0,"1717 San Sebastian Ln, Houston, TX 77058",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1695508302,/home/user/DataCenter/HAR_data/house_images/00...,5448,36


In [13]:
len(raw_df)

258960

In [14]:
def read_json(json_path):
    with open(json_path) as f:
        return json.load(f)

processed_df = []
n_rows = len(raw_df)
batch_size = 5000
n_batches = (n_rows + batch_size - 1)// batch_size

for batch_i in tqdm(range(n_batches)):
    sample_df = raw_df.iloc[batch_i * batch_size: (batch_i + 1) * batch_size]
    sample_df["content"] = sample_df["json_path"].apply(read_json)
    sample_df["LAT"], sample_df["LON"] = zip(*sample_df["content"].apply(lambda x: get_lon_lat(x["coords"][0], x["coords"][1])))
    sample_df["building_sqft2"] = sample_df["content"].apply(lambda x: parse_built_Sqft(x))
    sample_df["building_sqft"] = sample_df.apply(lambda x: x["building_sqft2"] if np.isnan(x["building_sqft2"]) is False else x["building_sqft"], axis=1)
    sample_df["Lot Size"] = sample_df["content"].apply(lambda x: parse_lot_size(x)[0])
    sample_df["Year Built"] = sample_df["content"].apply(lambda x: parse_built_year(x)[0])
    sample_df["Garage Number"] = sample_df["content"].apply(lambda x: get_garage_num(x))
    sample_df["Bedrooms"] = sample_df["content"].apply(lambda x: parse_bedroom(x))
    sample_df["Baths"] = sample_df["content"].apply(lambda x: parse_bath(x))
    sample_df["Maintenance Fee"] = sample_df["content"].apply(lambda x: parse_maintenance_fee(x))
    sample_df["Tax Rate"] = sample_df["content"].apply(lambda x: extract_tax_rate_tax_table(x["house_tax_table"]))
    sample_df["tax_rate2"] = sample_df["content"].apply(get_tax_rate)
    sample_df.loc[sample_df["Tax Rate"].isnull(), "Tax Rate"] = sample_df.loc[sample_df["Tax Rate"].isnull(), "tax_rate2"]
    sample_df["Recent Market Value"] = sample_df["content"].apply(lambda x: get_recent_market_value(x))
    sample_df["Recent Tax Value"] = sample_df["content"].apply(lambda x: get_recent_tax_value(x))
    sample_df["Property Type"] = sample_df["content"].apply(lambda x: parse_property_type(x))
    sample_df["County"] = sample_df["content"].apply(lambda x: x["house_features"]["County:"] if "County:" in x["house_features"] else None)
    sample_df["Private Pool"] = sample_df["content"].apply(lambda x: private_pool_feature(x))
    sample_df["Area Pool"] = sample_df["content"].apply(lambda x: area_pool_feature(x))
    sample_df["Private_pool_desc"] = sample_df["content"].apply(lambda x: private_pool_desc_feature(x))
    sample_df["Pool_feature_multiclass"] = sample_df["content"].apply(lambda x: private_pool_multiclass_feature(x))
    sample_df["Foundation_multiclass"] = sample_df["content"].apply(lambda x: Foundation_multiclass_feature(x))
    sample_df["Garage Types_multiclass"] = sample_df["content"].apply(lambda x: Garage_Types_multiclass_features(x))
    sample_df["Roof Type_multiclass"] = sample_df["content"].apply(lambda x: Roof_Type_multiclass(x))
    sample_df["floor_type_multiclass"] = sample_df["content"].apply(lambda x: floor_type_multiclass(x))
    sample_df["Exterior Type_multiclass"] = sample_df["content"].apply(lambda x: exterior_type_multiclass(x))
    sample_df["Exterior_multiclass"] = sample_df["content"].apply(lambda x: exterior_multiclass(x))
    sample_df["Style_multiclass"] = sample_df["content"].apply(lambda x: style_multiclass(x))
    sample_df["finance_option_multiclass"] = sample_df["content"].apply(lambda x: finanace_option_multiclass(x))
    sample_df["high_school_star"] = sample_df["content"].apply(lambda x: extract_high_school_name(x["house_schools"])[2] if "house_schools" in x else np.nan)
    sample_df["middle_school_star"] = sample_df["content"].apply(lambda x: extract_mid_school_name(x["house_schools"])[2] if "house_schools" in x else np.nan)
    sample_df["elementary_school_star"] = sample_df["content"].apply(lambda x: extract_elemetary_school_name(x["house_schools"])[2] if "house_schools" in x else np.nan)
    sample_df["school_org"] = sample_df["content"].apply(lambda x: school_org(x))
    sample_df["general_desc"] = sample_df["content"].apply(lambda x: x["general_desc"] if "general_desc" in x else None)
    processed_df.append(sample_df)

  0%|                                                                                                                                                                            | 0/52 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [05:36<00:00,  6.48s/it]


In [15]:
processed_final_df = pd.concat(processed_df)
len(processed_final_df), len(raw_df)

(258960, 258960)

In [16]:
# def get_neighborhood_median_value(content):
#     if "house_neighborhood" in content:
#         house_neighborhood = content["house_neighborhood"]
#     else:
#         house_neighborhood = {}
    
#     if "Median Appraised Value:" in house_neighborhood:
#         return eval(house_neighborhood["Median Appraised Value:"].replace("$", "").replace(",", ""))
#     elif "Average Price:" in house_neighborhood:
#         return  eval(house_neighborhood["Average Price:"].replace("$", "").replace(",", ""))
#     elif "Average List Price:" in house_neighborhood:
#         return  eval(house_neighborhood["Average List Price:"].replace("$", "").replace(",", ""))
#     else:
#         return np.nan

# def get_neighborhood_median_p_per_sqrt(x):
#     if "house_neighborhood" in x:
#         return x["house_neighborhood"]
#     else:
#         return None
# all_keys = []
# for keys in processed_final_df["content"].apply(lambda x: get_neighborhood_median_value(x)):
#     all_keys.extend(keys)
# np.unique(all_keys)
# sample_df["content"].apply(lambda x: get_neighborhood_median_value(x)).isna().sum(), len(sample_df)

In [17]:
# all_keys = []
# n_cnt = 0
# for keys in sample_df["content"].apply(lambda x: list(x["house_neighborhood"].keys()) if "house_neighborhood" in x else []):
#     all_keys.extend(keys)
#     if keys:
#         n_cnt += 1
# np.unique(all_keys), n_cnt

In [18]:
processed_final_df.head(2)

,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date,json_path,time_epoch,image_path,json_size,image_cnt,content,LAT,LON,...,Baths,Maintenance Fee,Tax Rate,tax_rate2,Recent Market Value,Recent Tax Value,Property Type,County,Private Pool,Area Pool,Private_pool_desc,Pool_feature_multiclass,Foundation_multiclass,Garage Types_multiclass,Roof Type_multiclass,floor_type_multiclass,Exterior Type_multiclass,Exterior_multiclass,Style_multiclass,finance_option_multiclass,high_school_star,middle_school_star,elementary_school_star,school_org,general_desc
57,"4513 Refugio Dr, Plano, TX 75024",https://www.har.com//homedetail/4513-refugio-d...,"4 bedrooms 3,395 Sqft. ($233/Sqft.) 3 full & 1...",Karla Davis Fathom Realty,789990.0,66.0,Under Contract - P,75024,4.0,0.0,0.0,3395.0,0.0,0.0,0.0,"4513 Refugio Dr, Plano, TX 75024",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1690411643,/home/user/DataCenter/HAR_data/house_images/00...,4425,12,"{'basic_info': [], 'coords': ['-96.78200500000...",-96.782005,33.102204,...,3.5,65.083333,1.864,NaN,157533.0,157533.0,single family,Collin County,No,None,None,[],[Slab],[Attached],[composition],"[carpet, ceramic tile, wood]","[brick, rock/stone]",[covered patio/porch],[traditional],[],NaN,5.0,5.0,[FRISCO ISD],"MOVE IN READY! Welcome to The Bartlet, this ge..."
8,"1516 Bay Area Blvd P12, Houston, TX 77058",https://www.har.com//homedetail/1516-bay-area-...,1 bedrooms 684 Sqft. ($117/Sqft.) 1 full baths...,Laurie Long RE/MAX Space Center,79900.0,79.0,Under Contract - PS,77058,1.0,0.0,0.0,684.0,0.0,0.0,0.0,"1516 Bay Area Blvd P12, Houston, TX 77058",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1689375459,/home/user/DataCenter/HAR_data/house_images/00...,5661,11,"{'basic_info': [], 'coords': ['-95.11409800000...",-95.114098,29.562197,...,1.0,272.000000,2.444,2.4437,69024.0,47052.0,townhouse/condo,Harris County,No,Yes,None,[],[Slab],[],[composition],"[tile, vinyl]","[brick, stucco]","[balcony, partially fenced, storage]",[traditional],"[cash sale, conventional, fha, investor, va]",5.0,NaN,4.0,[CLEAR CREEK ISD],This is the perfect starter home! EVERYTHING U...


In [19]:
processed_final_df["address_key"].nunique()

211960

In [20]:
target = ["price"]
dense_features = ["LON", 
                "LAT", 
                "building_sqft", 
                "Lot Size", 
                "Year Built", 
                "Garage Number", 
                "Bedrooms", 
                "Baths", 
                "Maintenance Fee", 
                "Tax Rate", 
                "Recent Market Value", 
                "Recent Tax Value"]
cate = ["status", "Property Type", "County", "Private Pool", "Area Pool"]
time_col = ["date"]
cate_multi = ["Foundation_multiclass", "Garage Types_multiclass", 
            "Roof Type_multiclass", "Pool_feature_multiclass", "floor_type_multiclass", 
            "finance_option_multiclass", "Exterior Type_multiclass", "Exterior_multiclass", 
            "Style_multiclass"]
add_numerial_features = ["elementary_school_star", "middle_school_star", "high_school_star"]
add_cate_multi = ["school_org"] # ["school_names", "school_grades", "school_org"]
processed_final_df[target + dense_features + cate + time_col + cate_multi + add_numerial_features + add_cate_multi].sample(3)

,price,LON,LAT,building_sqft,Lot Size,Year Built,Garage Number,Bedrooms,Baths,Maintenance Fee,Tax Rate,Recent Market Value,Recent Tax Value,status,Property Type,County,Private Pool,Area Pool,date,Foundation_multiclass,Garage Types_multiclass,Roof Type_multiclass,Pool_feature_multiclass,floor_type_multiclass,finance_option_multiclass,Exterior Type_multiclass,Exterior_multiclass,Style_multiclass,elementary_school_star,middle_school_star,high_school_star,school_org
104,230000.0,30.176575,-97.753395,1352.0,119.0,1980.0,NaN,3.0,2.0,208.000000,1.809,285005.0,285005.0,Under Contract - OP,townhouse,Travis County,No,None,2024_11_06,[Slab],[],[composition],[],[laminate],[],"[hardiplank type, stone, stone veneer]",[lighting],[],3.0,0.0,4.0,[AUSTIN ISD]
76,599000.0,32.822874,-96.695970,2388.0,1093.0,1965.0,NaN,3.0,2.0,NaN,2.500,448050.0,448050.0,Under Contract - P,single family,Dallas County,No,None,2024_01_15,[Pillar/Post/Pier],[],[composition],[],"[laminate, tile]",[],"[brick, siding]","[rain gutters, lighting]",[traditional],NaN,NaN,NaN,[DALLAS ISD]
62,849900.0,29.641216,-98.303412,4214.0,0.0,NaN,3.0,4.0,4.5,93.333333,1.703,909180.0,909180.0,Under Contract - P,single family,Comal County,No,None,2024_07_30,[Slab],[Attached],[composition],[],"[carpet, tile, wood]","[conventional, fha, tvlb, va]","[brick, stone, stucco]",[],[],5.0,4.0,4.0,[COMAL ISD]


In [21]:
## Filter out the rows with duplicate address_key 
processed_final_df_filtered = processed_final_df.drop_duplicates(subset=["address_key"], keep="first")

In [28]:
# old_df = pd.read_pickle("data/property_structured_12162024.pkl")
# old_df.head(2)

In [27]:
# keys1 = processed_final_df_filtered["address_key"].values
# keys2 = old_df["address_key"].values
# np.sum(keys1 != keys2)

In [ ]:
# processed_final_df_filtered.drop(columns=["content"], inplace=True)
# processed_final_df_filtered.to_pickle("data/property_structured_12162024.pkl")

In [43]:
# processed_final_df["address_key"].nunique(), len(processed_final_df)

In [44]:
# pool_features = []
# for feat_ in  df["finance_option_multiclass"]:
#     if feat_ is not None:
#         pool_features.extend(feat_)
# np.unique(pool_features)

In [45]:
# pool_features = Counter()
# for feat_ in  sample_df["content"].apply(lambda x: finanace_option_multiclass(x)):
#     if feat_ is not None:
#         pool_features.update(feat_)
# # [_[0] for _ in pool_features.most_common(50)]
# # [_[0] for _ in pool_features.most_common(8)]
# pool_features.most_common(30)

In [46]:
#
# test_df = sample_df.join(df.set_index("address"), on="address", how="inner", rsuffix="_r")

In [47]:
# col = "County"
# filter = test_df.apply(lambda x: "%s"%(x[col])  != "%s"%(x[col + "_r"]), axis=1)
# filter.sum(), len(test_df)

In [48]:
# test_df["tem"] = test_df["content"].apply(lambda x: x["house_features"]["Year Built:"] if "Year Built:" in x["house_features"] else None)
# test_df[filter][[col, col + "_r"]].sample(5)


In [49]:
# all_house_features = set()
# for feat in sample_df["content"].apply(lambda x: x["house_features"]):
#     all_house_features.update(feat.keys())
# len(all_house_features)

In [50]:
# [_ for _ in all_house_features if "financ" in _.lower()]

In [51]:
# sample_df["content"].iloc[0]["house_schools"]